# pandas

> In-memory processing

In [ ]:
import json
from collections import namedtuple

import pandas as pd

In [ ]:
data_dir = './out/extracts/100k'
schema_path = './out/schema.json'
data_years = range(1997, 2000)

# Table schema

Data types should be taken from schema.json, but I have no access to that file right now. Will get it later or re-create.

In [ ]:
# Create simple schema
# notest
# ! requires IPython
cols = !head -1 {data_dir}/1997.csv
cols = cols[0].split(',')
int_cols = ['year', 'employees', 'sales', 'year_est', 'parent_employees', 'parent_sales']
float_cols = ['latitude', 'longitude']
assert all(c in cols for c in int_cols + float_cols)
schema = dict(fields=[])
for c in cols:
    if c in int_cols:
        t = 'integer'
    elif c in float_cols:
        t = 'number'
    else:
        t = 'string'
    schema['fields'].append({'name': c, 'type': t})
with open(schema_path, 'w') as f:
    json.dump(schema, f, indent=1)

In [ ]:
# export
def dtypes_from_schema(schema_path):
    """Read JSON table schema and convert it to 
    dictionary (col_name -> col_dtype) that can be passed to pandas CSV reader."""
    with open(schema_path) as f:
        schema = json.load(f)
    dtypes = dict()
    for field in schema['fields']:
        c = field['name']
        st = field['type']
        if st == 'integer':
            # Integer arrays are experimental feature in pandas 0.25
            dt = 'Int64'
        elif st == 'number':
            dt = 'float64'
        else:
            dt = 'str'
        dtypes[c] = dt
    return dtypes

In [ ]:
cols = ['company', 'abi', 'naics', 'state', 'employees', 'sales', 'latitude', 'longitude']
df[cols].head(20)

,company,abi,naics,state,employees,sales,latitude,longitude
0,DEVELOPMENT ASSOCIATES,316162510,23721005,MA,3,600,42.05636,-72.6512
1,H P HOOD INC,009617762,11212001,MA,170,124270,42.08605,-72.6174
2,VINCENT ROMEO PAINTING CONTRS,106186919,23832003,MA,10,570,42.07566,-72.6026
3,NaN,106186927,54131001,MA,3,291,42.04345,-72.6453
4,NaN,106187008,44314226,MA,1,65,42.05128,-72.6129
5,NaN,106187016,33991011,MA,1,108,42.08958,-72.6371
6,NaN,106187040,62121003,MA,4,272,42.08792,-72.6281
7,NaN,106187057,62111107,MA,3,330,42.07198,-72.5926
8,LOUIS PALLOTTA PAVING,106187073,23731010,MA,4,316,42.08102,-72.6371
9,NaN,106187099,54111002,MA,2,230,42.08739,-72.6283


# Repeated cross-section analysis

Every year can be processed one at a time.

In [ ]:
def comp_stats_like_a_pro(data, stats=None):
    """Flexible function to compute various summary statistics for given data.
    Data can be Series, DataFrame or either one groupped.
    Returns new DataFrame with stats in columns and variables and groups in rows."""

In [ ]:
fields = 'count count_na count_notna count_unique min max sum mean std p1 p25 p50 p75 p99'.split()
defaults = [pd.np.nan] * len(fields)
Stats = namedtuple('Stats', fields, defaults=defaults)

def comp_stats(s):
    """Return dictionary of stats computed from a Series."""
    count = len(s)
    na = s.isna().sum()
    if pd.api.types.is_numeric_dtype(s):
        q = s.quantile([0, 0.01, 0.25, 0.5, 0.75, 0.99, 1])
        num_stats = dict(min=q[0], max=q[1], sum=s.sum(), mean=s.mean(), std=s.std(),
                         p1=q[0.01], p25=q[0.25], p50=q[0.5], p75=q[0.75], p99=q[0.99])
    else:
        num_stats = dict()
    return Stats(count=count, count_na=na, count_notna=count - na, count_unique=s.nunique(), **num_stats)

In [ ]:
stats_by_year = {}
stat_cols = ['sales', 'employees', 'state', 'naics2']
for y in data_years:
    dt = dtypes_from_schema(schema_path)
    df = pd.read_csv(f'{data_dir}/{y}.csv', dtype=dt)
    df['naics2'] = df['naics'].str[:2]
    s = [comp_stats(df[c]) for c in stat_cols]
    s = pd.DataFrame(s, index=stat_cols)
    stats_by_year[y] = s
stats_by_year = pd.concat(stats_by_year).reorder_levels([1, 0]).sort_index()

In [ ]:
stats_by_year

count  count_na  count_notna  count_unique   min        max  \
employees 1997  100000      3613        96387           286   1.0     7870.0   
          1998  100000     18690        81310           306   1.0     7870.0   
          1999  100000     18057        81943           299   1.0     8000.0   
naics2    1997  100000       298        99702            25   NaN        NaN   
          1998  100000       263        99737            25   NaN        NaN   
          1999  100000       282        99718            25   NaN        NaN   
sales     1997  100000     14680        85320          3607  11.0  1240000.0   
          1998  100000     14033        85967          3652  11.0  1908000.0   
          1999  100000     12837        87163          3689  11.0  2446880.0   
state     1997  100000         0       100000             1   NaN        NaN   
          1998  100000         0       100000             1   NaN        NaN   
          1999  100000         0       100000             1   NaN        NaN   

                        sum         mean           std    p1    p25    p50  \
employees 1997    1124699.0    11.668576     66.914361   1.0    2.0    3.0   
          1998    1117049.0    13.738150     80.039735   1.0    2.0    3.0   
          1999    1144094.0    13.962071     90.687935   1.0    2.0    3.0   
naics2    1997          NaN          NaN           NaN   NaN    NaN    NaN   
          1998          NaN          NaN           NaN   NaN    NaN    NaN   
          1999          NaN          NaN           NaN   NaN    NaN    NaN   
sales     1997  142992021.0  1675.949613  12936.637652  27.0  140.0  305.0   
          1998  149911073.0  1743.821152  14988.197406  27.0  138.0  294.0   
          1999  150621584.0  1728.044973  16820.913875  27.0  138.0  294.0   
state     1997          NaN          NaN           NaN   NaN    NaN    NaN   
          1998          NaN          NaN           NaN   NaN    NaN    NaN   
          1999          NaN          NaN           NaN   NaN    NaN    NaN   

                  p75       p99  
employees 1997    7.0    145.00  
          1998    8.0    160.00  
          1999    8.0    160.00  
naics2    1997    NaN       NaN  
          1998    NaN       NaN  
          1999    NaN       NaN  
sales     1997  690.0  24688.60  
          1998  702.0  25178.88  
          1999  696.0  24540.00  
state     1997    NaN       NaN  
          1998    NaN       NaN  
          1999    NaN       NaN

In [ ]:
dfg = df.groupby('naics2')
dfg['sales'].apply(lambda c: pd.Series(comp_stats(c))).to_frame().unstack()

sales                                                         \
             0       1        2       3      4          5           6    
naics2                                                                   
11        325.0     0.0    325.0    82.0   42.0    10530.0    166613.0   
21         47.0     1.0     46.0    39.0   75.0   453123.0    572713.0   
22         63.0    11.0     52.0    41.0  189.0   432480.0   1555239.0   
23       8258.0    13.0   8245.0   688.0   19.0   328000.0   7828689.0   
31        473.0     8.0    465.0   261.0   31.0   358000.0   2616062.0   
32       1425.0    35.0   1390.0   639.0   20.0   436000.0   7239598.0   
33       2810.0   125.0   2685.0  1029.0   43.0   609000.0  12957989.0   
42       6113.0    86.0   6027.0  1621.0   36.0  2446880.0  40555512.0   
44      10528.0    73.0  10455.0  1109.0   22.0   138600.0  16211892.0   
45       4596.0    36.0   4560.0   604.0   31.0   130000.0   4102296.0   
48       1448.0     4.0   1444.0   339.0   38.0    32700.0   1404556.0   
49        348.0   259.0     89.0    47.0   84.0    94620.0    196181.0   
51       1687.0   371.0   1316.0   359.0   34.0   196200.0   2549173.0   
52       3409.0   666.0   2743.0   279.0   11.0  1240000.0  15454785.0   
53       3771.0    59.0   3712.0   325.0   39.0    37500.0   2436631.0   
54      11281.0   150.0  11131.0   704.0   11.0  1553500.0   8549280.0   
55         27.0    24.0      3.0     3.0  264.0    55000.0     55795.0   
56       3955.0    16.0   3939.0   460.0   19.0    92150.0   1783851.0   
61       2743.0  1610.0   1133.0   195.0   20.0    19570.0    417879.0   
62      12068.0   997.0  11071.0   465.0   20.0   616000.0  11321635.0   
71       1349.0   104.0   1245.0   212.0   31.0    57300.0    634987.0   
72       5342.0    33.0   5309.0   305.0   31.0    32800.0   2804412.0   
81      12429.0  2924.0   9505.0   757.0   22.0  1908000.0   8592045.0   
92       3969.0  3927.0     42.0    35.0   20.0    26950.0     82818.0   
99       1254.0  1249.0      5.0     5.0   54.0    35000.0     36045.0   

                                                                       \
                  7             8       9        10      11        12   
naics2                                                                  
11        512.655385   1019.977541   42.00   100.00   288.0    490.00   
21      12450.282609  66874.522967   75.45   429.75   927.5   1914.75   
22      29908.442308  81704.152228  237.45  1229.25  5415.0  14310.00   
23        949.507459   7522.292054   40.00   127.00   294.0    508.00   
31       5625.939785  22767.470510   36.00   180.00   537.0   2805.00   
32       5208.343885  18014.160554   68.00   312.00   936.0   3620.00   
33       4826.066667  28027.077695   53.00   243.00   729.0   2485.00   
42       6728.971628  38931.112528   82.26   622.50  1665.0   4045.00   
44       1550.635294   4732.261821   66.00   231.00   480.0   1042.50   
45        899.626316   4562.572448   49.00   104.00   196.0    444.00   
48        972.684211   2208.468332   40.00   120.00   328.0    830.00   
49       2204.280899  10290.564069   84.00   168.00   336.0    966.00   
51       1937.061550   8487.126673   64.00   216.00   525.0   1195.50   
52       5634.263580  31471.912336   86.00   228.50   516.0   3010.00   
53        656.419989   1692.648122   56.00   176.00   352.0    558.00   
54        768.060372  15900.755388   33.00   138.00   230.0    345.00   
55      18598.333333  31525.050743  269.34   397.50   531.0  27765.50   
56        452.869002   2227.757613   20.00    80.00   126.0    264.00   
61        368.825243   1155.817244   27.00   102.00   153.0    255.00   
62       1022.638876  10431.708299   42.00   231.00   330.0    330.00   
71        510.029719   2056.478639   31.00   147.00   232.0    360.00   
72        528.237333   1188.688537   31.00    93.00   234.0    527.00   
81        903.950026  20893.070504   27.00    65.00   108.0    255.00   
92       1971.857143   4642.594841   24

# Dynamic analysis

Growth rates that take two years simultaneously.

# Full panel analysis

Use all years to run a regression